# 3.2. Métricas de Avaliação para Classificadores
---

<img src="https://selecao-stg.letscode.com.br/favicon.png" style="position: absolute; top: 0.75rem; right: 2rem; width: 50px" />

### Roteiro da Aula

1. Métricas de Avaliação  
    a. Acurácia  
    b. Matriz de Confusão  
        - Precisão  
        - Sensibilidade (Recall)  
        - Especificidade  
    c. ROC    
        - Curve  
        - Score/Gini  

## Métricas de Avaliação de Modelos

**O modelo melhorou ao utilizar mais variáveis?**

### Acurácia

Também conhecida como taxa de acerto, essa medida de desempenho traz a proporção de acertos sobre o total de observações. Assumindo que, dado um conjunto de variáveis explicativas $x$ associados a um conjunto de variável resposta $y$ um modelo $M$ foi treinado, temos que a acurácia do modelo $M$ ($ac(M)$) pode ser descrita matematicamente como:

$$ac({M}) = \frac{1}{n}\sum_{i = 1}^{n}{I(y_{i} = M(x_i)) }$$

A taxa de acerto é um número limitado entre 0 e 1. Quanto maior for o seu valor, melhor é o modelo $M$.

De forma similar, podemos obter a taxa de erro com:

$$err({M}) = 1 - ac(M)$$

Nesse caso, quanto menor a taxa de erro, melhor o modelo $M$.


**Como calcular a acurácia do clf e do clf2?**

**Para pensar:** Faça um modelo usando `'Pclass', 'Sex', 'SibSp', 'Fare'`, um segundo modelo sem o `'Fare'` e os compare. O que acontece?

### Matriz de Confusão

Uma alternativa para visualizar o desempenho de um modelo é analisar sua matriz de confusão, a qual ilustra o número de predições corretas e incorretas para cada classe do modelo. As linhas dessa matriz representam as classes verdadeiras, enquanto as colunas representam as classes preditas pelo modelo. Logo, casa elemento $m_{ij}$ de uma matriz de confusão $M_{MC}$ apresenta o número de exemplos da classe $i$ classificados como classe $j$. Dessa forma, os elementos na diagonal principal indicam as classificações feitas de forma correta, enquanto os outros elementos são os classificados de forma incorreta.

<img src=https://diegonogare.net/wp-content/uploads/2020/04/matrizConfusao-600x381.png height="400" width="400">

No Sklearn, a notação muda um pouco:

<img src="https://static.packt-cdn.com/products/9781838555078/graphics/C13314_06_05.jpg" width=400>

Note que a diagonal principal são as observações que o modelo acertou! Temos:

- Verdadeiros Positivos (VP): classificação correta da classe positivo;
- Verdadeiros Negativos (VN): classificação correta da classe negativo;
- Falsos Positivos (FP, erro tipo I): correto: negativo. Previsto: positivo.
- Falsos Negativos (FN, erro tipo II): correto: positivo. Previsto: negativo.

Por meio dela, temos as medidas quantitativas de quais classes possuem maior dificuldade de serem corretamente classificadas, se existe alguma "confusão" recorrente entre duas classes e mais uma série de medidas quantitativas sobre o modelo (a ser visto mais adiante).

### Medidas de Desempenho Derivadas da Matriz de Confusão

Dado a matriz de confusão mostranda no item anterior, podemos extrair, entre outras, as seguintes medidas de desempenho.

#### Precisão (precision)

É a proporção de exemplos positivos classificados corretamente entre todos aqueles preditos como positivos pelo modelo $M$.

$$prec(M) = \frac{\textrm{VP}}{\textrm{VP + FP}}$$

Pode ser vista como uma medida de exatidão do modelo. Uma precisão de 1 para uma determinada classe $C_1$ significa que cada item predito como pertencene a essa classe de fato pertence ela; porém, não nos trás informações sobre as predições das classes $C_2$.

#### Sensibilidade (recall, TVP)

Taxa de acerto na classe positiva, também conhecida como revocação ou taxa de verdadeiros positivos (TVP).

$$sens(M) = \frac{\textrm{VP}}{\textrm{VP + FN}}$$


$$\textrm{TVP}(M) = sens(M)$$

Pode ser vista como uma medida de completude do modelo. Uma sensibilidade de 1 para uma determinada classe $C_1$ significa que todos os itens que deveriam ser previstos como tal, de fato foram; mas não nos trás informações sobre as outras predições erradas dentro da própria classes $C_1$.


#### Especificidade

Taxa de acerto na classe negativa, sendo o complementar a taxa de falsos positivos (TFP).

$$esp(M) = \frac{\textrm{VN}}{\textrm{VN + FP}}$$


$$\textrm{TFP}(M) = 1 - esp(M) = \frac{\textrm{FP}}{\textrm{VN + FP}}$$

### ROC AUC

Uma forma alternativa e comum de avaliar classificadores em problemas binários é por meio do uso das curvas ROC (*Receiving Operating Characteristics*). Seu gráfico é bidimensional, no qual o eixo X está a TFP e no Y a TVP. Na próxima figura, temos um exemplo desse tipo de análise. Se um modelo se encontra na diagonal, dizemos que ele possui comportamento similar ao lançamento de uma moeda não viciada. Modelos abaixo dessa linha são piores que o aleatório, enquanto que acima são modelos melhores que o aleatório. Se um modelo está na ponta superior esquerda, chamada de céu ROC, dizemos que é um modelo perfeito; se está na ponta superior direita ou inferior esqueda, o modelo sempre classificará novos itens como positivos ou negativos, respectivamente; se está na ponta inferior direita, chamada de inferno ROC, esse modelo estará sempre errando. Um fator importante de ser notado é que se um modelo de classificação binária está *sempre* errado, basta invertermos sua predição para que seu desempenho perfeito.

<center><img src="https://s3-sa-east-1.amazonaws.com/lcpi/f6554123-1ee9-4838-8105-e0e32ab4c412.png" style="width: 300px;"/></center>

Apesar dessa análise gerar uma boa visualização para comparação de diferentes modelos, o processO mais usual é gerar uma curva ROC. Tomemos o seguinte exempplo: apesar da variável resposta ser binária em uma Regressão Logística, sua resposta é dado em um valor *contínuo* entre 0 e 1, que depois é aplicado um limiar de corte para definir se aquele caso pertence a classe positiva ou negativa; logo, temos um valor de TVP e TFP para cada ponto limiar, gerando assim uma curva para cada modelo de classificação, no formato das curvas na próxima figura.

<center><img src="https://s3-sa-east-1.amazonaws.com/lcpi/f1b7f22d-1f3b-460c-b53f-7f605e5d1bc9.png" style="width: 300px;"/></center>

Quando não há interseções entre as curvas de dois modelos, signica que o modelo que possui sua curva mais próxima do céu ROC é o que oferece melhor desempenho. Ao existir cruzamentos, cada um terá um desempenho melhor que o outro de acordo com a região. Entrentao, o mais comum é trazer a **área abaixo da curva ROC (AUC-ROC)** para cada modelo e compará-los com essa medida única, que é compreendida enre 0 e 1. Valores próximos de 1 são considerados os melhores; valores próximos a 0,5 são considerados aleatórios.

**A ROC AUC (*Area Under the Curve*) representa o grau ou medida de separabilidade. Diz o quanto o modelo é capaz de distinguir entre as classes. Quanto maior a AUC, melhor será o modelo em prever a classe 0 como sendo 0, e a classe 1 como sendo 1. Por analogia, quanto maior a AUC, melhor será a capacidade do modelo de distinguir entre transações fraudulentas e não fraudulentas.**

- Qual seria o valor ideal para a ROC AUC?

|   Valor do Score ROC AUR  | Poder de Classificação |
|:----------------:|:----------------------:|
|        $0.5$       |         Não há         |
| $0.7 \leq AUR < 0.8$ |        Aceitável       |
| $0.8 \leq AUR < 0.9$ |        Muito bom       |
|      $ \geq 0.9$      |        Excelente       |

Fonte: [Tese sobre Detecção de Fraude - Página 86](https://www.teses.usp.br/teses/disponiveis/45/45134/tde-01022016-204144/publico/Paulo_Oliveira_Mestrado_PPGCC.pdf)

## Problemas **Multiclasse**

#### Glass Dataset

Atributos:

1. **Id number:** 1 to 214
2. **RI:** refractive index
3. **Na:** Sodium (unit measurement: weight percent in corresponding oxide, as are attributes 4-10)
4. **Mg:** Magnesium
5. **Al:** Aluminum
6. **Si:** Silicon
7. **K:** Potassium
8. **Ca:** Calcium
9. **Ba:** Barium
10. **Fe:** Iron
11. **Type of glass:**
    - 1: building_windows_float_processed
    - 2: building_windows_non_float_processed
    - 3: vehicle_windows_float_processed
    - 4: vehicle_windows_non_float_processed (none in this database)
    - 5: containers
    - 6: tableware
    - 7: headlamps

Unidade de medida para os atributos 3 ao 10: `weight percent in corresponding oxide`

Para datasets pequenos, o ideal é utilizar o `solver='liblinear'`. Outra alternativa seria aumentar o número de iterações (`max_iter`). Em alguns casos, será necessário combinar as duas alternativas para que o modelo possa convergir e, dessa forma, obter os coeficientes corretos da regressão logística.

#### Para aprofundar sobre cálculos de métricas para problemas multiclasse

> https://towardsdatascience.com/comprehensive-guide-on-multiclass-classification-metrics-af94cfb83fbd#:~:text=ROC%20AUC%20score%20for%20multiclass,to%20distinguish%20between%20each%20class.

#### **Para Praticar**

Aplique a regressão logística ao conjunto de dados acima, e procure classificar, com base nas especificações do vidro, qual seria o tipo do vidro com aquelas especificações.

Em seguida, aplique a métrica da acurácia para avaliar o seu modelo.

**Para pensar:** É possível obter uma matriz de confusão para problemas multiclasse? E as métricas que extraímos a partir da matriz de confusão? E o ROC AUC/Gini?

### Exercícios

1. Carregue o dataset da iris (do Seaborn, se preferir) e utilize o `petal_length` e `sepal_length` para criar um modelo que classifique essa flor de acordo com as suas três espécies (`setosa`, `versicolor` e `virginica`). Em seguida, calcule as métricas que vimos e plote a região de decisão.

    Em seguida, divida também em dados de treino e teste (utilizando o `train_test_split`) para validar o modelo com dados que não foram utilizados durante o treino. Calcule as métricas, novamente, para treino e teste.
    
2. Faça o mesmo que foi solicitado na questão anterior; dessa vez, utilize o dataset do `breast_cancer`.
    > Dica: Importe o `load_breast_cancer` do submódulo `datasets` do `sklearn`.